<a href="https://colab.research.google.com/github/roy-sub/Time-Series-Air-Quality-Index-Prediction-and-Analysis-using-Multiple-Deep-Learning-Models/blob/main/Data_Pre_Processing_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras import optimizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'seaborn'

In [6]:
df = pd.read_csv('China.csv')
df

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,95.0,12.5,1013.5,-16.2,0.0,NW,2.4,Aotizhongxin
35060,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,81.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Aotizhongxin
35061,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,58.0,10.8,1014.2,-13.3,0.0,NW,1.1,Aotizhongxin
35062,35063,2017,2,28,22,21.0,44.0,12.0,87.0,700.0,35.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Aotizhongxin


In [7]:
df.shape

(35064, 18)

In [8]:
df.describe()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM
count,35064.000000,35064.000000,35064.000000,35064.000000,35064.000000,34139.000000,34346.000000,34129.000000,34041.000000,33288.000000,33345.000000,35044.000000,35044.000000,35044.000000,35044.000000,35050.000000
mean,17532.500000,2014.662560,6.522930,15.729637,11.500000,82.773611,110.060391,17.375901,59.305833,1262.945145,56.353358,13.584607,1011.846920,3.123062,0.067421,1.708496
std,10122.249256,1.177213,3.448752,8.800218,6.922285,82.135694,95.223005,22.823017,37.116200,1221.436236,57.916327,11.399097,10.404047,13.688896,0.910056,1.204071
min,1.000000,2013.000000,1.000000,1.000000,0.000000,3.000000,2.000000,0.285600,2.000000,100.000000,0.214200,-16.800000,985.900000,-35.300000,0.000000,0.000000
25%,8766.750000,2014.000000,4.000000,8.000000,5.750000,22.000000,38.000000,3.000000,30.000000,500.000000,8.000000,3.100000,1003.300000,-8.100000,0.000000,0.900000
50%,17532.500000,2015.000000,7.000000,16.000000,11.500000,58.000000,87.000000,9.000000,53.000000,900.000000,42.000000,14.500000,1011.400000,3.800000,0.000000,1.400000
75%,26298.250000,2016.000000,10.000000,23.000000,17.250000,114.000000,155.000000,21.000000,82.000000,1500.000000,82.000000,23.300000,1020.100000,15.600000,0.000000,2.200000
max,35064.000000,2017.000000,12.000000,31.000000,23.000000,898.000000,984.000000,341.000000,290.000000,10000.000000,423.000000,40.500000,1042.000000,28.500000,72.500000,11.200000


In [9]:
df = df.dropna()

In [10]:
df["PM10_24hr_avg"] = df["PM10"].rolling(window = 24, min_periods = 16).mean().values
df["PM2.5_24hr_avg"] = df["PM2.5"].rolling(window = 24, min_periods = 16).mean().values
df["SO2_24hr_avg"] = df["SO2"].rolling(window = 24, min_periods = 16).mean().values
df["NO2_24hr_avg"] = df["NO2"].rolling(window = 24, min_periods = 16).mean().values
df["CO_8hr_max"] = df["CO"].rolling(window = 8, min_periods = 1).max().values
df["O3_8hr_max"] = df["O3"].rolling(window = 8, min_periods = 1).max().values

C:\Users\monee\AppData\Local\Temp\ipykernel_11044\1222351727.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PM10_24hr_avg"] = df["PM10"].rolling(window = 24, min_periods = 16).mean().values
C:\Users\monee\AppData\Local\Temp\ipykernel_11044\1222351727.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PM2.5_24hr_avg"] = df["PM2.5"].rolling(window = 24, min_periods = 16).mean().values
C:\Users\monee\AppData\Local\Temp\ipykernel_11044\1222351727.py:3: SettingWithCopyWarning: 
A value is trying to

In [11]:
df

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,RAIN,wd,WSPM,station,PM10_24hr_avg,PM2.5_24hr_avg,SO2_24hr_avg,NO2_24hr_avg,CO_8hr_max,O3_8hr_max
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,...,0.0,NNW,4.4,Aotizhongxin,NaN,NaN,NaN,NaN,300.0,77.0
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,...,0.0,N,4.7,Aotizhongxin,NaN,NaN,NaN,NaN,300.0,77.0
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,...,0.0,NNW,5.6,Aotizhongxin,NaN,NaN,NaN,NaN,300.0,77.0
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,...,0.0,NW,3.1,Aotizhongxin,NaN,NaN,NaN,NaN,300.0,77.0
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,...,0.0,N,2.0,Aotizhongxin,NaN,NaN,NaN,NaN,300.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,...,0.0,NW,2.4,Aotizhongxin,32.583333,19.958333,10.125000,47.750000,400.0,122.0
35060,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,...,0.0,WNW,0.9,Aotizhongxin,30.916667,18.375000,10.041667,44.791667,500.0,122.0
35061,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,...,0.0,NW,1.1,Aotizhongxin,29.041667,16.833333,10.083333,42.375000,700.0,122.0
35062,35063,2017,2,28,22,21.0,44.0,12.0,87.0,700.0,...,0.0,NNW,1.2,Aotizhongxin,26.458333,15.291667,9.875000,40.458333,700.0,122.0


In [12]:
df.isnull().sum()

No                 0
year               0
month              0
day                0
hour               0
PM2.5              0
PM10               0
SO2                0
NO2                0
CO                 0
O3                 0
TEMP               0
PRES               0
DEWP               0
RAIN               0
wd                 0
WSPM               0
station            0
PM10_24hr_avg     15
PM2.5_24hr_avg    15
SO2_24hr_avg      15
NO2_24hr_avg      15
CO_8hr_max         0
O3_8hr_max         0
dtype: int64

In [13]:
df = df.dropna()
df.isnull().sum()

No                0
year              0
month             0
day               0
hour              0
PM2.5             0
PM10              0
SO2               0
NO2               0
CO                0
O3                0
TEMP              0
PRES              0
DEWP              0
RAIN              0
wd                0
WSPM              0
station           0
PM10_24hr_avg     0
PM2.5_24hr_avg    0
SO2_24hr_avg      0
NO2_24hr_avg      0
CO_8hr_max        0
O3_8hr_max        0
dtype: int64

In [14]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,RAIN,wd,WSPM,station,PM10_24hr_avg,PM2.5_24hr_avg,SO2_24hr_avg,NO2_24hr_avg,CO_8hr_max,O3_8hr_max
15,16,2013,3,1,15,8.0,15.0,7.0,14.0,400.0,...,0.0,NNE,2.8,Aotizhongxin,6.500000,4.437500,10.312500,17.812500,500.0,77.0
16,17,2013,3,1,16,9.0,19.0,9.0,13.0,400.0,...,0.0,NNW,3.9,Aotizhongxin,7.235294,4.705882,10.235294,17.529412,400.0,77.0
17,18,2013,3,1,17,10.0,23.0,11.0,15.0,400.0,...,0.0,NNE,2.8,Aotizhongxin,8.111111,5.000000,10.277778,17.388889,400.0,77.0
18,19,2013,3,1,18,11.0,20.0,8.0,20.0,500.0,...,0.0,NNE,2.1,Aotizhongxin,8.736842,5.315789,10.157895,17.526316,500.0,77.0
19,20,2013,3,1,19,8.0,14.0,12.0,30.0,500.0,...,0.0,N,2.8,Aotizhongxin,9.000000,5.450000,10.250000,18.150000,500.0,77.0


In [15]:
def get_PM25_subindex(x):
    if x <= 30:
        return x * 50 / 30
    elif x <= 60:
        return 50 + (x - 30) * 50 / 30
    elif x <= 90:
        return 100 + (x - 60) * 100 / 30
    elif x <= 120:
        return 200 + (x - 90) * 100 / 30
    elif x <= 250:
        return 300 + (x - 120) * 100 / 130
    elif x > 250:
        return 400 + (x - 250) * 100 / 130
    else:
        return 0

In [16]:
def get_PM10_subindex(x):
    if x <= 50:
        return x
    elif x <= 100:
        return x
    elif x <= 250:
        return 100 + (x - 100) * 100 / 150
    elif x <= 350:
        return 200 + (x - 250)
    elif x <= 430:
        return 300 + (x - 350) * 100 / 80
    elif x > 430:
        return 400 + (x - 430) * 100 / 80
    else:
        return 0

In [17]:
def get_SO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 380:
        return 100 + (x - 80) * 100 / 300
    elif x <= 800:
        return 200 + (x - 380) * 100 / 420
    elif x <= 1600:
        return 300 + (x - 800) * 100 / 800
    elif x > 1600:
        return 400 + (x - 1600) * 100 / 800
    else:
        return 0

In [18]:
def get_NO2_subindex(x):
    if x <= 40:
        return x * 50 / 40
    elif x <= 80:
        return 50 + (x - 40) * 50 / 40
    elif x <= 180:
        return 100 + (x - 80) * 100 / 100
    elif x <= 280:
        return 200 + (x - 180) * 100 / 100
    elif x <= 400:
        return 300 + (x - 280) * 100 / 120
    elif x > 400:
        return 400 + (x - 400) * 100 / 120
    else:
        return 0

In [19]:
def get_CO_subindex(x):
    if x <= 1:
        return x * 50 / 1
    elif x <= 2:
        return 50 + (x - 1) * 50 / 1
    elif x <= 10:
        return 100 + (x - 2) * 100 / 8
    elif x <= 17:
        return 200 + (x - 10) * 100 / 7
    elif x <= 34:
        return 300 + (x - 17) * 100 / 17
    elif x > 34:
        return 400 + (x - 34) * 100 / 17
    else:
        return 0

In [20]:
def get_O3_subindex(x):
    if x <= 50:
        return x * 50 / 50
    elif x <= 100:
        return 50 + (x - 50) * 50 / 50
    elif x <= 168:
        return 100 + (x - 100) * 100 / 68
    elif x <= 208:
        return 200 + (x - 168) * 100 / 40
    elif x <= 748:
        return 300 + (x - 208) * 100 / 539
    elif x > 748:
        return 400 + (x - 400) * 100 / 539
    else:
        return 0

In [21]:
df["PM2.5_SubIndex"] = df["PM2.5_24hr_avg"].apply(lambda x: get_PM25_subindex(x))
df["PM10_SubIndex"] = df["PM10_24hr_avg"].apply(lambda x: get_PM10_subindex(x))
df["SO2_SubIndex"] = df["SO2_24hr_avg"].apply(lambda x: get_SO2_subindex(x))
df["NO2_SubIndex"] = df["NO2_24hr_avg"].apply(lambda x: get_NO2_subindex(x))
df["CO_SubIndex"] = df["CO_8hr_max"].apply(lambda x: get_CO_subindex(x))
df["O3_SubIndex"] = df["O3_8hr_max"].apply(lambda x: get_O3_subindex(x))

C:\Users\monee\AppData\Local\Temp\ipykernel_11044\309051585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PM2.5_SubIndex"] = df["PM2.5_24hr_avg"].apply(lambda x: get_PM25_subindex(x))
C:\Users\monee\AppData\Local\Temp\ipykernel_11044\309051585.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["PM10_SubIndex"] = df["PM10_24hr_avg"].apply(lambda x: get_PM10_subindex(x))
C:\Users\monee\AppData\Local\Temp\ipykernel_11044\309051585.py:3: SettingWithCopyWarning: 
A value is trying to be set on a co

In [22]:
df

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,SO2_24hr_avg,NO2_24hr_avg,CO_8hr_max,O3_8hr_max,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NO2_SubIndex,CO_SubIndex,O3_SubIndex
15,16,2013,3,1,15,8.0,15.0,7.0,14.0,400.0,...,10.312500,17.812500,500.0,77.0,7.395833,6.500000,12.890625,22.265625,3141.176471,77.000000
16,17,2013,3,1,16,9.0,19.0,9.0,13.0,400.0,...,10.235294,17.529412,400.0,77.0,7.843137,7.235294,12.794118,21.911765,2552.941176,77.000000
17,18,2013,3,1,17,10.0,23.0,11.0,15.0,400.0,...,10.277778,17.388889,400.0,77.0,8.333333,8.111111,12.847222,21.736111,2552.941176,77.000000
18,19,2013,3,1,18,11.0,20.0,8.0,20.0,500.0,...,10.157895,17.526316,500.0,77.0,8.859649,8.736842,12.697368,21.907895,3141.176471,77.000000
19,20,2013,3,1,19,8.0,14.0,12.0,30.0,500.0,...,10.250000,18.150000,500.0,77.0,9.083333,9.000000,12.812500,22.687500,3141.176471,77.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,...,10.125000,47.750000,400.0,122.0,33.263889,32.583333,12.656250,59.687500,2552.941176,132.352941
35060,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,...,10.041667,44.791667,500.0,122.0,30.625000,30.916667,12.552083,55.989583,3141.176471,132.352941
35061,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,...,10.083333,42.375000,700.0,122.0,28.055556,29.041667,12.604167,52.968750,4317.647059,132.352941
35062,35063,2017,2,28,22,21.0,44.0,12.0,87.0,700.0,...,9.875000,40.458333,700.0,122.0,25.486111,26.458333,12.343750,50.572917,4317.647059,132.352941


In [23]:
## AQI bucketing
def get_AQI_bucket(x):
    if x <= 50:
        return "Good"
    elif x <= 100:
        return "Satisfactory"
    elif x <= 200:
        return "Moderate"
    elif x <= 300:
        return "Poor"
    elif x <= 400:
        return "Very Poor"
    elif x > 400:
        return "Severe"
    else:
        return np.NaN

df["Checks"] = (df["PM2.5_SubIndex"] > 0).astype(int) + \
                (df["PM10_SubIndex"] > 0).astype(int) + \
                (df["SO2_SubIndex"] > 0).astype(int) + \
                (df["NO2_SubIndex"] > 0).astype(int) + \
                (df["CO_SubIndex"] > 0).astype(int) + \
                (df["O3_SubIndex"] > 0).astype(int)

df["AQI_calculated"] = round(df[["PM2.5_SubIndex", "PM10_SubIndex", "SO2_SubIndex", "NO2_SubIndex",
                                 "CO_SubIndex", "O3_SubIndex"]].max(axis = 1))
df.loc[df["PM2.5_SubIndex"] + df["PM10_SubIndex"] <= 0, "AQI_calculated"] = np.NaN
df.loc[df.Checks < 3, "AQI_calculated"] = np.NaN

df["AQI_bucket_calculated"] = df["AQI_calculated"].apply(lambda x: get_AQI_bucket(x))
df[~df.AQI_calculated.isna()].head(13)

C:\Users\monee\AppData\Local\Temp\ipykernel_11044\3563561489.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Checks"] = (df["PM2.5_SubIndex"] > 0).astype(int) + \
C:\Users\monee\AppData\Local\Temp\ipykernel_11044\3563561489.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["AQI_calculated"] = round(df[["PM2.5_SubIndex", "PM10_SubIndex", "SO2_SubIndex", "NO2_SubIndex",
C:\Users\monee\AppData\Local\Temp\ipykernel_11044\3563561489.py:30: SettingWithCopyWarning: 
A value is trying to be set on a 

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,O3_8hr_max,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NO2_SubIndex,CO_SubIndex,O3_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
15,16,2013,3,1,15,8.0,15.0,7.0,14.0,400.0,...,77.0,7.395833,6.500000,12.890625,22.265625,3141.176471,77.0,6,3141.0,Severe
16,17,2013,3,1,16,9.0,19.0,9.0,13.0,400.0,...,77.0,7.843137,7.235294,12.794118,21.911765,2552.941176,77.0,6,2553.0,Severe
17,18,2013,3,1,17,10.0,23.0,11.0,15.0,400.0,...,77.0,8.333333,8.111111,12.847222,21.736111,2552.941176,77.0,6,2553.0,Severe
18,19,2013,3,1,18,11.0,20.0,8.0,20.0,500.0,...,77.0,8.859649,8.736842,12.697368,21.907895,3141.176471,77.0,6,3141.0,Severe
19,20,2013,3,1,19,8.0,14.0,12.0,30.0,500.0,...,77.0,9.083333,9.000000,12.812500,22.687500,3141.176471,77.0,6,3141.0,Severe
20,21,2013,3,1,20,11.0,17.0,13.0,33.0,600.0,...,77.0,9.523810,9.380952,12.976190,23.571429,3729.411765,77.0,6,3729.0,Severe
21,22,2013,3,1,21,12.0,18.0,16.0,35.0,500.0,...,77.0,10.000000,9.772727,13.295455,24.488636,3729.411765,77.0,6,3729.0,Severe
22,23,2013,3,1,22,15.0,19.0,21.0,57.0,700.0,...,76.0,10.652174,10.173913,13.858696,26.521739,4317.647059,76.0,6,4318.0,Severe
23,24,2013,3,1,23,24.0,24.0,26.0,54.0,600.0,...,76.0,11.875000,10.750000,14.635417,28.229167,4317.647059,76.0,6,4318.0,Severe
24,25,2013,3,2,0,22.0,24.0,24.0,44.0,500.0,...,74.0,13.125000,11.583333,15.677083,30.156250,4317.647059,74.0,6,4318.0,Severe


In [24]:
df[~df.AQI_calculated.isna()].AQI_bucket_calculated.value_counts()

AQI_bucket_calculated
Severe    31800
Name: count, dtype: int64

In [25]:
df

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,O3_8hr_max,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NO2_SubIndex,CO_SubIndex,O3_SubIndex,Checks,AQI_calculated,AQI_bucket_calculated
15,16,2013,3,1,15,8.0,15.0,7.0,14.0,400.0,...,77.0,7.395833,6.500000,12.890625,22.265625,3141.176471,77.000000,6,3141.0,Severe
16,17,2013,3,1,16,9.0,19.0,9.0,13.0,400.0,...,77.0,7.843137,7.235294,12.794118,21.911765,2552.941176,77.000000,6,2553.0,Severe
17,18,2013,3,1,17,10.0,23.0,11.0,15.0,400.0,...,77.0,8.333333,8.111111,12.847222,21.736111,2552.941176,77.000000,6,2553.0,Severe
18,19,2013,3,1,18,11.0,20.0,8.0,20.0,500.0,...,77.0,8.859649,8.736842,12.697368,21.907895,3141.176471,77.000000,6,3141.0,Severe
19,20,2013,3,1,19,8.0,14.0,12.0,30.0,500.0,...,77.0,9.083333,9.000000,12.812500,22.687500,3141.176471,77.000000,6,3141.0,Severe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,35060,2017,2,28,19,12.0,29.0,5.0,35.0,400.0,...,122.0,33.263889,32.583333,12.656250,59.687500,2552.941176,132.352941,6,2553.0,Severe
35060,35061,2017,2,28,20,13.0,37.0,7.0,45.0,500.0,...,122.0,30.625000,30.916667,12.552083,55.989583,3141.176471,132.352941,6,3141.0,Severe
35061,35062,2017,2,28,21,16.0,37.0,10.0,66.0,700.0,...,122.0,28.055556,29.041667,12.604167,52.968750,4317.647059,132.352941,6,4318.0,Severe
35062,35063,2017,2,28,22,21.0,44.0,12.0,87.0,700.0,...,122.0,25.486111,26.458333,12.343750,50.572917,4317.647059,132.352941,6,4318.0,Severe


In [26]:
df.describe()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,...,CO_8hr_max,O3_8hr_max,PM2.5_SubIndex,PM10_SubIndex,SO2_SubIndex,NO2_SubIndex,CO_SubIndex,O3_SubIndex,Checks,AQI_calculated
count,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,...,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.000000,31800.0,31800.000000
mean,17907.863208,2014.715723,6.403491,15.585440,11.626981,81.900264,109.169358,17.170334,58.995629,1256.988208,...,1715.315252,85.963471,158.442129,99.395838,21.114304,72.559050,10290.089715,96.462517,6.0,10290.092956
std,10053.291361,1.162213,3.416854,8.791325,6.927144,80.450765,93.713305,22.765453,37.058620,1223.440427,...,1490.905720,68.326843,123.759916,61.535936,22.865209,32.909511,8770.033649,86.588217,0.0,8770.036759
min,16.000000,2013.000000,1.000000,1.000000,0.000000,3.000000,2.000000,0.571200,2.000000,100.000000,...,100.000000,1.000000,7.395833,5.500000,2.500000,3.229167,788.235294,1.000000,6.0,788.000000
25%,9427.750000,2014.000000,3.000000,8.000000,6.000000,22.000000,38.000000,3.000000,30.000000,500.000000,...,700.000000,35.000000,53.385417,52.416667,5.625000,49.010417,4317.647059,35.000000,6.0,4318.000000
50%,18100.500000,2015.000000,6.000000,15.000000,12.000000,58.000000,86.000000,8.000000,53.000000,800.000000,...,1200.000000,71.000000,112.916667,91.791667,12.552083,67.968750,7258.823529,71.000000,6.0,7259.000000
75%,26594.250000,2016.000000,9.000000,23.000000,18.000000,114.000000,154.000000,21.000000,82.000000,1500.000000,...,2100.000000,117.000000,265.982639,130.305556,26.614583,91.718750,12552.941176,125.000000,6.0,12553.000000
max,35064.000000,2017.000000,12.000000,31.000000,23.000000,713.000000,948.000000,229.000000,290.000000,10000.000000,...,10000.000000,423.000000,566.442308,465.677083,127.013889,220.083333,59023.529412,339.888683,6.0,59024.000000


In [27]:
df.to_csv(r'air_quality_data_preprocessed', index = False)